# explore sql database

## issues

- db Postgres not sqlite
- db contains role app_ncse2 and errors arise in db creation if this is not considered
- invalid command error `psql:/media/jonno/ncse/ncse_webapp_db.sql:2137: error: invalid command`
- when set up to read the database from python you need to either set up a password and use it or change the pg_hba file to allow for 'peer' password free connections


- periodicals_issue seems to be the link to the pdf
- what is the periodicals_page?
- the ocr appears to be stored in  periodicals_article


In [2]:
import pandas as pd
import tiktoken
import psycopg2
from sqlalchemy import create_engine, inspect
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Set up sql connection using sql alchemy


The table 'periodicals_articletype' gives the article types used by article_type_id as
- Article
- Ad
- Picture

The table 'periodicals_publication' gives basic statistics about each periodical

column 'publication_id' on table 'periodicals_issue' links to column 'id' on table 'periodicals_publication'

column 'issue_id' on table 'periodical_page' links to column 'id' on table 'periodicals_issue

table 'periodicals_article' links to tables 'periodicals_page'  and 'periodicals_issue' using the columns 'issue_id' and 'page_id' which both map to column 'id' on thier respective tables


In [3]:

# Your database credentials and connection details
database_name = "ncse"
user = "jonno"

# Create the SQLAlchemy engine for a local connection without specifying host or port
engine = create_engine(f'postgresql://{user}@/{database_name}')

In [4]:
inspector = inspect(engine)

table_names = inspector.get_table_names()
print(table_names)



['auth_user_user_permissions', 'cms_indexpage', 'cms_richtextpage', 'django_admin_log', 'periodicals_articletype', 'django_migrations', 'auth_permission', 'auth_group', 'django_content_type', 'auth_user_groups', 'auth_user', 'cms_homepage', 'periodicals_article', 'taggit_taggeditem', 'periodicals_issuecomponent', 'periodicals_issue', 'periodicals_page', 'periodicals_publication', 'taggit_tag', 'wagtailcore_collectionviewrestriction_groups', 'wagtailcore_groupcollectionpermission', 'wagtailcore_pageviewrestriction_groups', 'wagtaildocs_document', 'wagtailforms_formsubmission', 'wagtailembeds_embed', 'wagtailimages_image', 'wagtailimages_rendition', 'wagtailcore_page', 'wagtailcore_grouppagepermission', 'wagtailcore_pagerevision', 'wagtailcore_pageviewrestriction', 'wagtailcore_site', 'wagtailusers_userprofile', 'auth_group_permissions', 'wagtailcore_collection', 'wagtailcore_collectionviewrestriction', 'wagtailredirects_redirect']


In [23]:

# Define your SQL query
query = f"SELECT * FROM periodicals_publication;"


# Use read_sql_query to execute the query and store the result in a DataFrame
periodicals_publication_df = pd.read_sql_query(query, engine)

periodicals_publication_df.to_parquet('./data/periodicals_publication.parquet')

periodicals_publication_df

,id,abbreviation,slug,title,description,ordering,title_image,year_from,year_to,issue_count,page_count,article_count,headnotes_url,weekly
0,27,NS,ns,Northern Star (1837-1852),,1837,northern-star.jpg,1837,1852,2201,17853,230596,/headnotes/nss.html,True
1,20,L,l,Leader (1850-1860),,1850,leader.jpg,1850,1860,1011,24456,162431,/headnotes/ldr.html,True
2,19,T,t,Tomahawk (1867-1870),,1867,tomahawk.jpg,1867,1870,188,2125,12825,/headnotes/ttw.html,True
3,26,PC,pc,Publishers’ Circular (1880-1890),,1880,publishers-circular.jpg,1880,1890,285,20827,56739,/headnotes/tec.html,True
4,24,EWJ,ewj,English Woman’s Journal (1858-1864),,1858,english-womens-journal.jpg,1858,1864,91,5663,8061,/headnotes/ewj.html,False
5,22,MRUC,mruc,Monthly Repository (1806-1838) and Unitarian C...,,1806,monthly-respository.jpg,1806,1837,487,26783,51202,/headnotes/mrp.html,False


In [24]:

# Define your SQL query
query = f"SELECT * FROM periodicals_issue;"


# Use read_sql_query to execute the query and store the result in a DataFrame
periodicals_issue_df = pd.read_sql_query(query, engine)

periodicals_issue_df.to_parquet('./data/periodicals_issue.parquet')

periodicals_issue_df


,id,uid,slug,issue_date,number_of_pages,pdf,publication_id,component_id,edition,article_count
0,2764,EMRP_02/12/1810,emrp_02121810,1810-12-02,19,periodicals/041-MRP-1810-12-02-PG001-SINGLE_ia...,22,4.0,1,74
1,4608,NS2_01/11/1845,ns2_01111845,1845-11-01,8,periodicals/101-NS2-1845-11-01-PG001-SINGLE_RR...,27,NaN,2,130
2,3386,VM2-NCSEPRODUCT:2109,vm2-ncseproduct2109,1855-10-06,24,periodicals/101-LDR-1855-10-06-001-SINGLE_9oNu...,20,NaN,1,135
3,3237,VM2-NCSEPRODUCT:2047,vm2-ncseproduct2047,1854-07-15,24,periodicals/101-LDR-1854-07-15-001-SINGLE_bQxA...,20,NaN,1,157
4,3095,VM2-NCSEPRODUCT:2009,vm2-ncseproduct2009,1853-10-22,24,periodicals/101-LDR-1853-10-22-001-SINGLE_6MnW...,20,NaN,Town,174
...,...,...,...,...,...,...,...,...,...,...
4258,4583,TTW_23/05/1868,ttw_23051868,1868-05-23,10,periodicals/041-TTW-1868-05-23-PG001-SINGLE_0x...,19,NaN,1,57
4259,4587,TTW_19/09/1868,ttw_19091868,1868-09-19,12,periodicals/041-TTW-1868-09-19-PG001-SINGLE_Re...,19,NaN,1,61
4260,4584,TTW_26/09/1868,ttw_26091868,1868-09-26,12,periodicals/041-TTW-1868-09-26-PG001-SINGLE_OI...,19,NaN,1,65
4261,4585,TTW_05/09/1868,ttw_05091868,1868-09-05,12,periodicals/041-TTW-1868-09-05-PG001-SINGLE_Re...,19,NaN,1,51


In [25]:
# Define your SQL query
query = f"SELECT ID, HEIGHT, NUMBER, IMAGE, WIDTH, ISSUE_ID, ARTICLE_COUNT, LABEL FROM periodicals_page;"


# Use read_sql_query to execute the query and store the result in a DataFrame
periodicals_page_df = pd.read_sql_query(query, engine)

periodicals_page_df.to_parquet('./data/periodicals_page.parquet')

periodicals_page_df

,id,height,number,image,width,issue_id,article_count,label
0,170269,2270,6,periodicals/101-NS5-1842-10-29-PG001-SINGLE/Pg...,1579,5747,9,6
1,79360,1659,15,periodicals/101-CLD-1853-01-22-001-SINGLE/Pg01...,1169,2308,2,None
2,81343,1752,6,periodicals/101-CLD-1850-05-04-001-SINGLE/Pg00...,1314,2390,5,None
3,81344,1737,7,periodicals/101-CLD-1850-05-04-001-SINGLE/Pg00...,1322,2390,5,None
4,81345,1741,8,periodicals/101-CLD-1850-05-04-001-SINGLE/Pg00...,1326,2390,2,None
...,...,...,...,...,...,...,...,...
97694,160863,2001,11,periodicals/041-TTW-1868-09-19-PG001-SINGLE_Re...,1768,4587,7,123
97695,160864,2001,12,periodicals/041-TTW-1868-09-19-PG001-SINGLE_Re...,1768,4587,7,124
97696,160870,1934,6,periodicals/041-TTW-1868-09-19-PG001-SINGLE_Re...,1768,4587,1,unpag
97697,160871,1934,7,periodicals/041-TTW-1868-09-19-PG001-SINGLE_Re...,1768,4587,0,unpag


In [38]:
# Define your SQL query
query = f"SELECT * FROM periodicals_article LIMIT 10;"


# Use read_sql_query to execute the query and store the result in a DataFrame
chunk = pd.read_sql_query(query, engine)

chunk#.to_parquet('./data/periodicals_page.parquet')

,id,aid,slug,position_in_page,title,description,content,content_html,bounding_box,title_image,article_type_id,continuation_from_id,continuation_to_id,issue_id,page_id,is_department
0,863754,Ad00212,ad00212,24.0,,,NoneON THE CONCEALED CAUSE THAT PREYS ON THE H...,"<p class=""article-content"">ON THE CONCEALED CA...","{'x0': '514', 'x1': '884', 'y0': '1707', 'y1':...",periodicals/101-NS2-1848-02-12-PG001-SINGLE/Ad...,2,NaN,NaN,5023,164348,False
1,422496,Ad02309,ad02309,142.0,,,NoneSOILED LTNENS . — Manufacturers' Sam pel P...,"<p class=""article-content"">SOILED LTNENS . — M...","{'x0': '858', 'x1': '1262', 'y0': '90', 'y1': ...",periodicals/101-CLD-1850-05-04-001-SINGLE/Ad02...,2,NaN,NaN,2390,81337,False
2,857644,Ar00510,ar00510,90.0,WOLVER1UMPTOX. The Ti.y-1'i.atr WoRKF.RS...,,NoneWOLVER 1 UMPTOX . The Ti . y-1 ' i . atr W...,"<p class=""article-content"">WOLVER 1 UMPTOX . T...","{'x0': '1545', 'x1': '1915', 'y0': '3361', 'y1...",periodicals/101-NS2-1851-07-05-PG001-SINGLE/Ar...,1,NaN,857645.0,4979,163999,False
3,857645,Ar00511,ar00511,91.0,WOLVER1UMPTOX. The Ti.y-1'i.atr WoRKF.RS...,,"None,-eomniiltal against three of them—viz ., ...","<p class=""article-content"">,-eomniiltal agains...","{'x0': '1930', 'x1': '2308', 'y0': '142', 'y1'...",,1,857644.0,NaN,4979,163999,False
4,921377,Ar00201,ar00201,NaN,Untitled Article,,NoneBEJGHTOX *—The following wu omitted in oar...,"<p class=""article-content"">BEJGHTOX *—The foll...","{'x0': '67', 'x1': '305', 'y0': '130', 'y1': '...",periodicals/101-NS3-1841-07-03-PG001-SINGLE/Ar...,1,NaN,921378.0,5440,167809,False
5,863755,Ad00213,ad00213,25.0,,,NoneA BERNETHY'S PILL for the KERVES and MUSJT...,"<p class=""article-content"">A BERNETHY'S PILL f...","{'x0': '509', 'x1': '887', 'y0': '3019', 'y1':...",periodicals/101-NS2-1848-02-12-PG001-SINGLE/Ad...,2,NaN,863756.0,5023,164348,False
6,921378,Ar00202,ar00202,NaN,Untitled Article,,Noneout the Bcripture to the Tery letter—namel...,"<p class=""article-content"">out the Bcripture t...","{'x0': '301', 'x1': '535', 'y0': '99', 'y1': '...",,1,921377.0,NaN,5440,167809,False
7,426155,Ar02000,ar02000,113.0,TURKEY—ITS HISTORY ANI> PROGRESS. From t...,,Nonepressed as they were —I suspect by his Maj...,"<p class=""article-content"">pressed as they wer...","{'x0': '29', 'x1': '541', 'y0': '85', 'y1': '9...",,1,426146.0,NaN,2411,81838,False
8,408344,Ar01005,ar01005,57.0,"SATURDAY, JANUARY 22, 1853.",,"NoneSATURDAY , JANUARY 22 , 1853 .SATURDAY , J...","<p class=""article-content"">SATURDAY , JANUARY ...","{'x0': '30', 'x1': '386', 'y0': '229', 'y1': '...",periodicals/101-CLD-1853-01-22-001-SINGLE/Ar01...,1,NaN,NaN,2308,79355,False
9,972349,Ar00501,ar00501,NaN,Untitled Article,,"NoneSociiL ^ ES TlvjLL i—On Sunday , a large r...","<p class=""article-content"">SociiL ^ ES TlvjLL ...","{'x0': '298', 'x1': '544', 'y0': '90', 'y1': '...",,1,972348.0,972350.0,5819,170844,False


In [33]:
periodicals_publication_df

,id,abbreviation,slug,title,description,ordering,title_image,year_from,year_to,issue_count,page_count,article_count,headnotes_url,weekly
0,27,NS,ns,Northern Star (1837-1852),,1837,northern-star.jpg,1837,1852,2201,17853,230596,/headnotes/nss.html,True
1,20,L,l,Leader (1850-1860),,1850,leader.jpg,1850,1860,1011,24456,162431,/headnotes/ldr.html,True
2,19,T,t,Tomahawk (1867-1870),,1867,tomahawk.jpg,1867,1870,188,2125,12825,/headnotes/ttw.html,True
3,26,PC,pc,Publishers’ Circular (1880-1890),,1880,publishers-circular.jpg,1880,1890,285,20827,56739,/headnotes/tec.html,True
4,24,EWJ,ewj,English Woman’s Journal (1858-1864),,1858,english-womens-journal.jpg,1858,1864,91,5663,8061,/headnotes/ewj.html,False
5,22,MRUC,mruc,Monthly Repository (1806-1838) and Unitarian C...,,1806,monthly-respository.jpg,1806,1837,487,26783,51202,/headnotes/mrp.html,False


# Extract and clean from sql database

The database is too large for my computer to hold in RAM so I am loading it by chunks and saving into parquet format dropping the raw text field which can have duplicated text in it. Dropping the text also allows me to reduce the size

In [44]:

enc = tiktoken.encoding_for_model('gpt-3.5-turbo')

#pattern used to identify wierd symbols
pattern = r'[:;"\-_\+\*\^\|¦\'!,\\/»«\]\[]'  

# Connection parameters
database_name = "ncse"
user = "jonno"
engine = create_engine(f'postgresql://{user}@/{database_name}')

# Define your SQL query without loading the 'content' column
columns = "ID, AID, SLUG, POSITION_IN_PAGE, TITLE, DESCRIPTION, CONTENT_HTML, ARTICLE_TYPE_ID, CONTINUATION_FROM_ID, CONTINUATION_TO_ID, ISSUE_ID, PAGE_ID, IS_DEPARTMENT"
query = f"SELECT {columns} FROM periodicals_article;"

# Specify the chunk size
chunk_size = 100000  # Adjust based on your memory constraints

# Use a loop to process and save each chunk
chunk_number = 0
for chunk in pd.read_sql_query(query, engine, chunksize=chunk_size):
    # Define the filename for each chunk using the specified naming pattern
    filename = f'./data/ncse_text_chunks/ncse_base_{chunk_number}.parquet'

    # remove html tags from text
    chunk['content_html'] = chunk['content_html'].str.replace(r'(<p class="article-content">)|(<p class="article-header">)', '', regex=True).str.replace("</p>\n", "", regex=True)

    #add the token count to work out what I need to do
    chunk['total_tokens'] = chunk['content_html'].apply(lambda x:len(enc.encode(x)))

    # Count occurrences of the pattern in each row of 'content_html'
    chunk['symbol_count'] = chunk['content_html'].str.count(pattern)
    chunk['symbol_fract'] = chunk['symbol_count']/chunk['total_tokens']

    #add in information on date, and publication. This makes understanding the distributions easier
    chunk = chunk.merge(periodicals_issue_df.loc[:, ['id', 'issue_date', 'publication_id']].set_index('id'), how = 'left', left_on='issue_id', right_index=True)
    # Save the chunk to a Parquet file
    chunk.to_parquet(filename, index=False)
    
    # Increment the chunk number for the next file
    chunk_number += 1


# End

This takes a up a lot of ram and is not relevant to the analysis, so all firther work can take place in an other notebook.